In [ ]:
# Imports

from pathlib import Path
import os
import pandas as pd
import numpy as np
import importlib.util
import time
import matplotlib.pyplot as plt

In [ ]:
# Time window
START_DATE = '2025-01-01'
END_DATE = '2025-07-01'

# Simulation controls
N_SIMULATIONS = 1000000   
TOP_K_PLOTS = 10        
TOP_PERCENTILE = 0.01 

# Risk free rate (annual)
RF_ANNUAL = 0.04

# Rebalancing mode, 'static' or 'random_daily'
REBALANCE_MODE = 'random_daily'

# Outputs
OUT_DIR = Path.cwd()
(OUT_DIR).mkdir(parents=True, exist_ok=True)
print(f"Output directory: {OUT_DIR}")
print(f"Rebalance mode: {REBALANCE_MODE}")

In [ ]:
# Locate data dir
from pathlib import Path
data_dir = Path.cwd().parent.parent / 'data'
prices_csv = data_dir / 'etf_prices.csv'
prices = pd.read_csv(prices_csv, index_col=0, parse_dates=True)

# Slice by window and compute returns
prices = prices.loc[START_DATE:END_DATE]
if prices.empty:
    raise ValueError("No price data in the selected window.")

returns = prices.pct_change().dropna()
asset_list = returns.columns.tolist()
print(f"Assets: {len(asset_list)} -> {asset_list}")
print(f"Window: {returns.index.min().date()} -> {returns.index.max().date()} (n={len(returns)})")

In [ ]:
TRADING_DAYS = 252

def max_drawdown(series: pd.Series) -> float:
    cum = (1 + series).cumprod()
    peak = cum.cummax()
    dd = (cum / peak) - 1.0
    return dd.min()

def portfolio_metrics(port_ret: pd.Series, rf_annual: float = 0.0) -> dict:
    if len(port_ret) == 0:
        return {"cagr": np.nan, "vol": np.nan, "sharpe": np.nan, "mdd": np.nan}
    # CAGR
    wealth = (1 + port_ret).cumprod()
    n_years = len(port_ret) / TRADING_DAYS
    cagr = wealth.iloc[-1] ** (1 / n_years) - 1 if n_years > 0 else np.nan
    # Volatility (annualized)
    vol = port_ret.std(ddof=1) * np.sqrt(TRADING_DAYS)
    # Sharpe (excess return over rf)
    rf_daily = rf_annual / TRADING_DAYS
    excess = port_ret - rf_daily
    sharpe = (excess.mean() * TRADING_DAYS) / vol if vol > 0 else np.nan
    # Max drawdown
    mdd = max_drawdown(port_ret)
    return {"cagr": cagr, "vol": vol, "sharpe": sharpe, "mdd": mdd}

def random_weights(n_assets: int) -> np.ndarray:
    w = np.random.rand(n_assets)
    s = w.sum()
    return (w / s) if s > 0 else np.ones(n_assets) / n_assets

def simulate_portfolios(returns: pd.DataFrame, n_sims: int, rf_annual: float = 0.0, mode: str = 'static') -> tuple[pd.DataFrame, pd.DataFrame]:
    n_assets = returns.shape[1]
    n_days = returns.shape[0]
    all_weights = np.zeros((n_sims, n_assets))
    metrics = []

    if mode == 'static':
        for i in range(n_sims):
            w = random_weights(n_assets)
            all_weights[i, :] = w
            port_ret = (returns.values @ w)
            port_ret = pd.Series(port_ret, index=returns.index)
            m = portfolio_metrics(port_ret, rf_annual=rf_annual)
            m["final_wealth"] = float((1 + port_ret).cumprod().iloc[-1])
            m["idx"] = i
            metrics.append(m)
    elif mode == 'random_daily':
        # For random_daily, weights change every day; final weights snapshot is the last day's weights
        for i in range(n_sims):
            daily_w = np.apply_along_axis(lambda _: random_weights(n_assets), 1, np.zeros((n_days, n_assets)))
            # Compute daily portfolio returns as row-wise dot of returns and weights
            port_ret_vals = (returns.values * daily_w).sum(axis=1)
            port_ret = pd.Series(port_ret_vals, index=returns.index)
            m = portfolio_metrics(port_ret, rf_annual=rf_annual)
            m["final_wealth"] = float((1 + port_ret).cumprod().iloc[-1])
            m["idx"] = i
            metrics.append(m)
            all_weights[i, :] = daily_w[-1]
    else:
        raise ValueError("Unsupported mode. Use 'static' or 'random_daily'.")

    metrics_df = pd.DataFrame(metrics)
    weights_df = pd.DataFrame(all_weights, columns=returns.columns)
    return metrics_df, weights_df

In [ ]:
# Run simulation

start_t = time.time()
metrics_df, weights_df = simulate_portfolios(returns, N_SIMULATIONS, rf_annual=RF_ANNUAL, mode=REBALANCE_MODE)
elapsed = time.time() - start_t

# Rank by Sharpe, then by final wealth

metrics_df = metrics_df.sort_values(["sharpe", "final_wealth"], ascending=[False, False]).reset_index(drop=True)
print(f"Simulations: {N_SIMULATIONS} in {elapsed:.2f}s | Mode: {REBALANCE_MODE}")
print("Top 10 portfolios (Sharpe):")
print(metrics_df.head(10))


In [ ]:
# Top percentile summary

cut = int(np.ceil(len(metrics_df) * (TOP_PERCENTILE / 100.0)))
cut = max(cut, 1)
summary = metrics_df.head(cut).agg({"cagr": ["mean", "max"], "vol": ["mean", "min"], "sharpe": ["mean", "max"], "mdd": ["mean", "min"], "final_wealth": ["mean", "max"]})
print(f"\nTop {TOP_PERCENTILE}% summary over {cut} portfolios:")
print(summary)

In [ ]:
# Plot wealth curves for top portfolios

k = min(TOP_K_PLOTS, len(metrics_df))
fig, ax = plt.subplots(figsize=(12, 6))

for i in range(k):
    sim_idx = int(metrics_df.loc[i, 'idx'])
    w = weights_df.iloc[sim_idx].values
    port_ret = pd.Series(returns.values @ w, index=returns.index)
    wealth = (1 + port_ret).cumprod()
    ax.plot(wealth, label=f"Sim {sim_idx} | Sharpe={metrics_df.loc[i,'sharpe']:.2f}")

ax.set_title(f"Top {k} Portfolio Wealth Curves ({START_DATE} to {END_DATE})")
ax.set_xlabel("Date")
ax.set_ylabel("Wealth (Start=1.0)")
ax.grid(True, alpha=0.3)
ax.legend(loc='upper left', fontsize=8)
plt.tight_layout()
plt.savefig(OUT_DIR / 'mc_wealth_topK.png', dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
# Scatter: CAGR vs Volatility (colored by Sharpe)
fig, ax = plt.subplots(figsize=(8, 6))
sc = ax.scatter(metrics_df['vol'], metrics_df['cagr'], c=metrics_df['sharpe'], cmap='viridis', s=12, alpha=0.7)
ax.set_xlabel('Volatility (annualized)')
ax.set_ylabel('CAGR')
ax.set_title('Monte Carlo Portfolios: Return vs Risk')
cb = plt.colorbar(sc, ax=ax)
cb.set_label('Sharpe')
plt.tight_layout()
plt.savefig(OUT_DIR / 'mc_scatter_return_vs_risk.png', dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
# Save a single combined CSV with metrics, weights, and flags

n = len(metrics_df)
k = min(TOP_K_PLOTS, n)
cut = int(np.ceil(n * (TOP_PERCENTILE / 100.0)))
cut = max(cut, 1)

# Align weights to the sorted metrics order using original simulation index
weights_sorted = weights_df.iloc[metrics_df['idx'].astype(int)].reset_index(drop=True)
combined = pd.concat([metrics_df.reset_index(drop=True), weights_sorted], axis=1)

# Add flags and metadata
combined.insert(0, 'rank', range(1, n + 1))
combined['is_topK'] = combined['rank'] <= k
combined['is_top_percentile'] = combined['rank'] <= cut
combined['mode'] = REBALANCE_MODE
combined['start_date'] = START_DATE
combined['end_date'] = END_DATE
combined['rf_annual'] = RF_ANNUAL

out_csv = OUT_DIR / 'mc_results.csv'
combined.to_csv(out_csv, index=False)
print(f"Saved single CSV: {out_csv}")

# Keep plots already saved above
print("Plots saved:", OUT_DIR / 'mc_wealth_topK.png', ',', OUT_DIR / 'mc_scatter_return_vs_risk.png')